In [11]:
import requests
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup

# specify the url
import urllib.request


In [12]:
#!/usr/bin/env python

try:
    # For Python 3.0 and later
    from urllib.request import urlopen
except ImportError:
    # Fall back to Python 2's urllib2
    from urllib2 import urlopen

import certifi
import json

def get_data(url, stock):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)




In [13]:
def get_income_data(url, stock):
    """
    Receive the content of ``url``, parse it as JSON and return the object.

    Parameters
    ----------
    url : str

    Returns
    -------
    dict
    """
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)



In [14]:
NIR = [2]
EBITDA = [10]
DTE = [.7]
EPS = [1.2]
GPR = [.7]
PTBV = [1]
PTS = [1]
prices = {}
api_key = "87f2035b04c9137524da80d59c6c7083"

In [15]:
def stockInfo(stock):
    print(stock)
    income_url = "https://financialmodelingprep.com/api/v3/income-statement/" + stock + "?apikey=" + api_key + "&limit=120"
    income_data = get_data(income_url, stock)
    balance_url = "https://financialmodelingprep.com/api/v3/balance-sheet-statement/" + stock + "?apikey=" + api_key + "&limit=120"
    balance_data = get_data(balance_url, stock)
    price_url = "https://financialmodelingprep.com/api/v3/historical-price-full/" + stock + "?apikey=" + api_key + "&limit=120"
    price_data = get_data(price_url, stock)
    price = price_data["historical"][0]["close"]
    liabilites = balance_data[0]['totalCurrentLiabilities']
    equity = balance_data[0]['totalStockholdersEquity']
    assets = balance_data[0]['totalAssets']
    shares = balance_data[0]['commonStock']
    revenue = income_data[0]['revenue']
    net_income_ratio = income_data[0]['netIncomeRatio']
    ebitda_ratio = income_data[0]['ebitdaratio']
    eps = income_data[0]['epsdiluted']
    gpr = income_data[0]['grossProfitRatio']
    price_to_book_value = price / ((assets - liabilites) / equity)
    price_to_sales = (shares * equity) / revenue
    debt_to_equity = liabilites/equity
    NIR.append(net_income_ratio)
    EBITDA.append(ebitda_ratio)
    DTE.append(debt_to_equity)
    EPS.append(eps)
    GPR.append(gpr)
    PTBV.append(price_to_book_value)
    PTS.append(price_to_sales)



In [16]:
def normalize(arr):
	mn = min(arr)
	mx = max(arr)
	normed = []
	for i in arr:
		standard = (i - mn)/(mx-mn)
		normed.append(standard)
	return normed

In [17]:
def score(stocks, ndte, nnir, nebitda, ngpr, neps, nptbv, npts):
    scores = {}
    for i in range(len(stocks)):
        ticker = stocks[i]
        print(ticker)
        dteScore = abs(ndte[0] - ndte[i + 1])
        nirScore = abs(nnir[0] - nnir[i+ 1])
        ebitdaScore = abs(nebitda[0] - nebitda[i+ 1])
        gprScore = abs(ngpr[0] - ngpr[i+ 1])
        epsScore = abs(neps[0] - neps[i+ 1])
        ptbvScore = abs(nptbv[0] - nptbv[i + 1])
        ptsScore = abs(npts[0] - npts[i+ 1])
        scoreVal = dteScore + nirScore + ebitdaScore + gprScore + epsScore + ptbvScore + ptsScore
        scores[ticker] = (scoreVal)
    return scores

In [18]:
large_cap_stocks = ["UNH", "JNJ", 'XOM', 'JPM', 'PG', 'LLY', 'CVX', 'PFE', 
          'ABBV', 'PEP', 'MRK', 'KO', 'BAC', 'WMT', 'AVGO', 'ABT', 'NEE', 'WFC', 'BMY', 'VZ', 'CSCO',
                    'COP', 'CMCSA','RTX' ,'HON',  'SCHW']
small_cap_stocks = ["TRGP", "ENTG", "WOLF", "FWONK", "TECH", "PTC", "NDSN", "TPL", "DAR", "ELS", 
                   "FICO", "RGEN", "NBIX", "GGG", "PCTY", "SWAV", "SRPT", "AMH", "TTC", "CUBE"]
for i in large_cap_stocks:
    stockInfo(i)

UNH


<ipython-input-12-16be51af4abb>:25: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


JNJ
XOM
JPM
PG
LLY
CVX
PFE
ABBV
PEP
MRK
KO
BAC
WMT
AVGO
ABT
NEE
WFC
BMY
VZ
CSCO
COP
CMCSA
RTX
HON
SCHW


In [19]:
ndte = normalize(DTE)
nnir = normalize(NIR)
nebitda = normalize(EBITDA)
ngpr = normalize(GPR)
neps = normalize(EPS)
nptbv = normalize(PTBV)
npts = normalize(PTS)

In [21]:
scores = score(large_cap_stocks, ndte, nnir, nebitda, ngpr, neps, nptbv, npts)
sortedScores = sorted(scores.items(), key=lambda x: x[1])
print(large_cap_stocks)
print(sortedScores)

UNH
JNJ
XOM
JPM
PG
LLY
CVX
PFE
ABBV
PEP
MRK
KO
BAC
WMT
AVGO
ABT
NEE
WFC
BMY
VZ
CSCO
COP
CMCSA
RTX
HON
SCHW
['UNH', 'JNJ', 'XOM', 'JPM', 'PG', 'LLY', 'CVX', 'PFE', 'ABBV', 'PEP', 'MRK', 'KO', 'BAC', 'WMT', 'AVGO', 'ABT', 'NEE', 'WFC', 'BMY', 'VZ', 'CSCO', 'COP', 'CMCSA', 'RTX', 'HON', 'SCHW']
[('KO', 2.1740878885652695), ('PFE', 2.2525999624045827), ('BMY', 2.2664600031405815), ('CMCSA', 2.2831507473074524), ('MRK', 2.3010989315368913), ('NEE', 2.319896025931339), ('CSCO', 2.3444292634520174), ('VZ', 2.3816031297479143), ('JNJ', 2.6555477820791706), ('ABT', 2.6788789068569794), ('PG', 2.7246950405709844), ('ABBV', 2.9553024943498243), ('PEP', 2.9742049422702035), ('LLY', 2.9854076272459715), ('COP', 3.085735183786241), ('SCHW', 3.158722870084598), ('WMT', 3.1937196992383803), ('RTX', 3.208750206655772), ('XOM', 3.2283392632885763), ('HON', 3.2310535666298015), ('WFC', 3.378893244267789), ('CVX', 3.477704752628158), ('AVGO', 3.53630400363364), ('JPM', 3.8793709369668226), ('BAC', 4.13564

In [22]:
def risk (x):
	url = "https://yh-finance.p.rapidapi.com/stock/v2/get-analysis"

	querystring = {"symbol":x,"region":"US"}

	headers = {
		"X-RapidAPI-Host": "yh-finance.p.rapidapi.com",
		"X-RapidAPI-Key": "ea0fe23a72mshbfc9dfafe36fc9dp1870e7jsnbb7a2a7713ff"
	}

	response = requests.request("GET", url, headers=headers, params=querystring)
	data = json.loads(response.text)
	try:
		beta = data['summaryDetail']['beta']['raw']
		print(beta)
		return beta
	except:
		return -1


In [24]:
keys = []
for i in range(0,6):
	keys.append(sortedScores[i][0])
print(keys)
totalbeta = 0
numBeta = 0
for i in keys:
	beta = risk(i)
	if(beta != -1):
		totalbeta += beta
		numBeta += 1
print(totalbeta/numBeta)



['KO', 'PFE', 'BMY', 'CMCSA', 'MRK', 'NEE']
0.572218
0.637067
0.364406
0.98305
0.328297
0.493367
0.5630674999999999
